In [1]:
import pandas as pd
import pandas_profiling as pp
import plotly.express as px
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from pyspark.sql import SparkSession

In [98]:
from pyspark.ml.feature import StringIndexer

insured_hobbies_indexer = StringIndexer(inputCol="insured_hobbies", outputCol="insured_hobbiesIndex")
df = insured_hobbies_indexer.fit(df).transform(df)



IllegalArgumentException: requirement failed: Output column insured_hobbiesIndex already exists.

In [99]:
onehotencoder_age_vector = OneHotEncoder(inputCol="insured_hobbiesIndex", outputCol="insured_hobbies_vec")
df = onehotencoder_age_vector.fit(df).transform(df)

In [56]:


name = 'insured_hobbies'
udf = UserDefinedFunction(lambda x :'Other' if x!='chess' or x!='cross-fit' else x)
new_df = df.withColumn('insured_hobbies', udf(df.insured_hobbies))

In [101]:
# new_df.select("insured_hobbies").sample(False, 0.1, seed=9).limit(20).show()
df.select('insured_hobbies_vec').distinct().collect()

[Row(insured_hobbies_vec=SparseVector(2, {0: 1.0})),
 Row(insured_hobbies_vec=SparseVector(2, {1: 1.0})),
 Row(insured_hobbies_vec=SparseVector(2, {}))]

In [69]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline

def onehot(df,cols):

    stages = []

    c=0

    for col in cols:
        str_encoder_cols = [x+"Index" for x in cols]
        indexer = StringIndexer(inputCol=col, outputCol=str_encoder_cols[c])
        stages.append(indexer)
        c+=1

    c=0
    for col in cols:
        ohe_cols = [x+"Vector" for x in cols]
        onehotencoder = OneHotEncoder(inputCol=str_encoder_cols[c], outputCol=ohe_cols[c])
        stages.append(onehotencoder)
        c+=1

    pipeline = Pipeline(stages=stages)

    df_transformed = pipeline.fit(df).transform(df)

    pipeline.write().overwrite().save('/pipeline')

    cols_drop = str_encoder_cols + cols

    df = df.drop(*cols_drop)

    return df_transformed
      

In [67]:
qualification_indexer = StringIndexer(inputCol="qualification", outputCol="qualificationIndex")
gender_indexer = StringIndexer(inputCol="gender", outputCol="genderIndex")
age_indexer = StringIndexer(inputCol="age", outputCol="ageIndex")
onehotencoder_qualification_vector = OneHotEncoder(inputCol="qualificationIndex", outputCol="qualification_vec")
onehotencoder_gender_vector = OneHotEncoder(inputCol="genderIndex", outputCol="gender_vec")
onehotencoder_age_vector = OneHotEncoder(inputCol="ageIndex", outputCol="age_vec")
#Create pipeline and pass all stages
pipeline = Pipeline(stages=[qualification_indexer,
                            gender_indexer,
                            age_indexer,
                            onehotencoder_qualification_vector,
                            onehotencoder_gender_vector,
                            onehotencoder_age_vector
                    ])

In [75]:
df_encoded = onehot(df,['incident_severity','insured_sex'])

['incident_severityIndex', 'insured_sexIndex']
['incident_severityVector', 'insured_sexVector']
['incident_severityIndex', 'insured_sexIndex']


In [77]:
df_encoded.select('insured_sexVector').show()

+-----------------+
|insured_sexVector|
+-----------------+
|        (1,[],[])|
|        (1,[],[])|
|    (1,[0],[1.0])|
|    (1,[0],[1.0])|
|        (1,[],[])|
|    (1,[0],[1.0])|
|        (1,[],[])|
|        (1,[],[])|
|    (1,[0],[1.0])|
|        (1,[],[])|
|    (1,[0],[1.0])|
|    (1,[0],[1.0])|
|        (1,[],[])|
|    (1,[0],[1.0])|
|    (1,[0],[1.0])|
|    (1,[0],[1.0])|
|        (1,[],[])|
|        (1,[],[])|
|    (1,[0],[1.0])|
|    (1,[0],[1.0])|
+-----------------+
only showing top 20 rows



In [45]:
import json
import time
import schedule
import http.client
import pandas as pd

In [46]:
from pyspark.sql import SparkSession

In [47]:
spark = SparkSession.builder.appName("test").getOrCreate()


In [74]:
df = spark.read.csv('./data/insurance_claims.csv', header=True, sep=',', inferSchema=True)


In [44]:
df_pandas = df.toPandas()

In [6]:
df_json = df_pandas.to_json()

In [10]:
# Serializing json 
json_object = json.dumps(df_json, indent = 4)
  
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
c = http.client.HTTPConnection('127.0.0.1', 8000)
data = pd.read_csv('Data_repository/Data/test.csv')
data = data.to_json()
c.request('POST', '/get_prediction_results', json.dumps(data))
result = c.getresponse().read()
prediction_result = json.loads(result)
prediction_result_df = pd.read_json(prediction_result)
print(prediction_result_df.head())
return prediction_result_df
    

if __name__ == '__main__':
    schedule.every(0).minutes.do(get_prediction_result)
    while True:
        schedule.run_pending()
        time.sleep(1)

In [7]:
data = '{"months_as_customer":{"0":328},"age":{"0":48},"policy_number":{"0":521585},"policy_bind_date":{"0":1413504000000},"policy_state":{"0":"OH"},"policy_csl":{"0":"250\\/500"},"policy_deductable":{"0":1000},"policy_annual_premium":{"0":1406.91},"umbrella_limit":{"0":0},"insured_zip":{"0":466132},"insured_sex":{"0":"MALE"},"insured_education_level":{"0":"MD"},"insured_occupation":{"0":"craft-repair"},"insured_hobbies":{"0":"sleeping"},"insured_relationship":{"0":"husband"},"capital-gains":{"0":53300},"capital-loss":{"0":0},"incident_date":{"0":1422144000000},"incident_type":{"0":"Single Vehicle Collision"},"collision_type":{"0":"Side Collision"},"incident_severity":{"0":"Major Damage"},"authorities_contacted":{"0":"Police"},"incident_state":{"0":"SC"},"incident_city":{"0":"Columbus"},"incident_location":{"0":"9935 4th Drive"},"incident_hour_of_the_day":{"0":5},"number_of_vehicles_involved":{"0":1},"property_damage":{"0":"YES"},"bodily_injuries":{"0":1},"witnesses":{"0":2},"police_report_available":{"0":"YES"},"total_claim_amount":{"0":71610},"injury_claim":{"0":6510},"property_claim":{"0":13020},"vehicle_claim":{"0":52080},"auto_make":{"0":"Saab"},"auto_model":{"0":"92x"},"auto_year":{"0":2004},"fraud_reported":{"0":"Y"}}'


In [8]:
a_json = json.loads(data)


In [9]:
df_pandas = pd.DataFrame.from_dict(a_json)
df_spark = spark.createDataFrame(df_pandas)



In [10]:
import sys
import argparse

def get_args():

    parser = argparse.ArgumentParser(description='Zeiss Task')


    parser.add_argument("--app_name", type=str, default="Zeiss_classification_task")
    parser.add_argument("--data_filename", type=str, default="./data/insurance_claims.csv",
                                    help="data file in csv format")
    parser.add_argument("--columns_to_drop", type=list, 
                        default=['policy_number','policy_bind_date','policy_state','insured_zip','incident_location',
                        'incident_date','auto_make','auto_model','insured_occupation','age', 'total_claim_amount'])
    parser.add_argument("--store_schema", type=bool, default=False)
    parser.add_argument("--schema_path", type=str, default="/src/data/feature_store/")
    parser.add_argument("--columns_to_encode", type=list, 
                        default=['policy_csl', 'insured_sex', 'insured_education_level','insured_hobbies', 'insured_relationship',
                        'incident_type', 'incident_severity','authorities_contacted', 'incident_state', 'incident_city','collision_type'])
    parser.add_argument("--preprocess_hobbies", type=bool, default=True)
    parser.add_argument("--tune_hyper_params", type=bool, default=False)
    parser.add_argument("--best_hyper_params_filepath", type=str, default='/src/data/best_hyper_params')
    parser.add_argument("--model_path", type=str, default='/src/model')
    parser.add_argument("--hyper_params", type=dict, 
                        default={'criterion': 'gini', 
                                    'max_depth': 5, 
                                    'min_samples_leaf': 2, 
                                    'min_samples_split': 2
                                    })
    parser.add_argument("--target", type=str, default='fraud_reported')

    return parser.parse_args()



In [11]:
sys.argv = sys.argv[10:]

In [12]:
args = get_args()

In [14]:
from src.utils.utils import  data_preprocessing

df_processed = data_preprocessing(df_spark, args)



AttributeError: 'OneHotEncoder' object has no attribute 'fit'

In [29]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer.load(os.getcwd() + args.schema_path + '/schemaData/' + 'incident_city')

pyspark.ml.feature.StringIndexer

In [33]:
import pickle
